In [ ]:
import os
import json
import pandas as pd
from pandas import json_normalize
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import randint

# WinRate Regression 
In this notebook, we'll aim to build a regression model for a certain composition

In [ ]:
dataset = pd.read_csv('csv/flattened_matches.csv')
df = dataset[dataset['queue_id']== 1100].copy() #ranked data only

column_key = list(df.keys())

column_key = [item for item in column_key if (   "trait_" not in item and "unit_" not in item  and item not in ['placement'])]
#print ( column_key )
df.drop(column_key,axis='columns',inplace=True)

column_key = list(df.keys())
column_key = [item for item in column_key if ( "_item" in item)]
df.drop(column_key,axis='columns',inplace=True)


print(list(df.keys())) #remaining columns

In [ ]:
# summonUnits = ['unit_TFT14_AnnieTibbers', 'unit_TFT14_SummonLevel2', 'unit_TFT14_SummonLevel4', 'unit_TFT14_Summon_Turret']
# df.drop(summonUnits,axis='columns',inplace=True)
# df = df.fillna(0).infer_objects()

In [ ]:

pd.set_option('display.max_columns', None)
df

In [ ]:
# #Quick Validation
df[df['unit_TFT14_Kobuko']==3.0]['placement'].mean() #Getting a single 3* kobuko will very likely win you the game, which is expected.



In [ ]:
df[(df['trait_TFT14_Suits'] == 3.0) & (df['placement'] == 1)]

In [ ]:
y = df['placement'].astype(float)
x = df.drop(columns='placement')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import cupy as cp  # GPU array library
import itertools
from tqdm import tqdm

x_gpu = cp.array(x)
y_gpu = cp.array(y)
dtrain = xgb.DMatrix(x_gpu, label=y_gpu)

xgb_model = xgb.XGBRegressor(
    tree_method='hist',  # still required
    device='cuda'        # tells XGBoost to run on GPU
)

param_grid = {
    'n_estimators': range( 100,2000,100),
    'learning_rate': np.arange(0.05,0.1,0.025),
    'max_depth': range( 5,50 , 10 ),
    'subsample': np.arange (0.1,1,0.4)
}

param_combinations = list(itertools.product(
    param_grid['n_estimators'],
    param_grid['learning_rate'],
    param_grid['max_depth'],
    param_grid['subsample']
))
# param_grid = {
#     'n_estimators': range( 100,600,100) ,
#     'max_depth': range( 6,12 , 2 ),
#     'learning_rate': np.arange(0.05,0.2,0.05),
#     'subsample': np.arange (0.1,1,0.2)
# }

# Step 3: Search
best_score = float("inf")
best_params = None

for n_estimators, learning_rate, max_depth, subsample in tqdm(param_combinations, desc="Grid Search"):
    params = {
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "device": "cuda",  # Make sure we are using GPU
        #"eta": eta,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "subsample": subsample,
        "n_estimators": n_estimators,
        "verbosity": 0,
        "nthread": -1  # or "n_jobs": -1
    }

    # Perform cross-validation on the current set of hyperparameters
    cv_results = xgb.cv(
        params=params,
        dtrain=dtrain,
        num_boost_round=10, #100
        nfold=3,  # 5-fold cross-validation
        metrics="rmse",
        early_stopping_rounds=10,
        seed=42,
        as_pandas=True
    )

    # Extract the best RMSE score from the cross-validation results
    score = cv_results["test-rmse-mean"].min()
    
    # Track the best score and corresponding parameters
    if score < best_score:
        best_score = score
        best_params = params

    #print(f"Test RMSE: {score:.4f} for params: {params}")

# Step 4: Train best model
print("\nBest Params:", best_params)
final_model = xgb.train(best_params, dtrain, num_boost_round=1000)

# Predict on CPU (move data back from GPU if needed)
y_pred = final_model.predict(xgb.DMatrix(x_gpu))
y_true = cp.asnumpy(y_gpu)      # Convert back to NumPy if needed
y_pred = cp.asnumpy(y_pred)

# Metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"\nModel Evaluation:")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R²:  {r2:.4f}")
# # Custom scoring (minimize MSE, so we negate it)
# scorer = make_scorer(mean_squared_error, greater_is_better=False)

# # Set up cross-validation
# cv = KFold(n_splits=5, shuffle=True, random_state=42)

# # Grid search
# grid_search = GridSearchCV(estimator=xgb_model, 
#                            param_grid=param_grid,
#                            scoring=scorer, 
#                            cv=cv,
#                            verbose=1,
#                            n_jobs=-1)

# grid_search.fit(x, y)

# Results
# print("Best Parameters:", grid_search.best_params_)
# print("Best MSE:", -grid_search.best_score_)

# best_xgb =grid_search.best_estimator_
# # Predict
# y_pred = best_xgb.predict(x_test)
# print("MAE:", mean_absolute_error(y_test, y_pred))
# print("MSE:", mean_squared_error(y_test, y_pred))
# print("R² Score:", r2_score(y_test, y_pred))

In [14]:

rf_param_grid = {
    'n_estimators': range(100,150,25) ,
    'max_depth': [45,50,55],
    'min_samples_split': [7,8,9,10,11],
    'min_samples_leaf': [1,2,4,6,8],
    'max_features' : ['sqrt', 'log2']
}

rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), 
                              rf_param_grid, cv=5, scoring='neg_mean_squared_error',
                              n_jobs=-1)

rf_grid_search.fit(x_train, y_train) 
model = rf_grid_search.best_estimator_

# param_dist = {
#     #'n_estimators': 125,
#     'max_depth': randint(35, 55),
#     'min_samples_split': randint(2, 10),
#     'min_samples_leaf': randint(1, 10),
#     'max_features': ['sqrt', 'log2', 0.5,0.6,0.7, 0.75, 0.8,0.9,1.0],
#     'bootstrap': [True]
# }

# rf = RandomForestRegressor(n_estimators=125, random_state=42)
# search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=150, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
# search.fit(x_train, y_train)
# model = search.best_estimator_

y_pred = model.predict(x_test) 

KeyboardInterrupt: 

In [ ]:
rf_grid_search.best_params_, model.get_params()

In [ ]:
y_pred = model.predict(x_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

# ({'max_depth': 50, 'n_estimators': 125},
#  {'bootstrap': True,
#   'ccp_alpha': 0.0,
#   'criterion': 'squared_error',
#   'max_depth': 44,
#   'max_features': 'log2',
#   'max_leaf_nodes': None,
#   'max_samples': None,
#   'min_impurity_decrease': 0.0,
#   'min_samples_leaf': 1,
#   'min_samples_split': 9,
#   'min_weight_fraction_leaf': 0.0,
#   'monotonic_cst': None,
#   'n_estimators': 260,
#   'n_jobs': None,
#   'oob_score': False,
#   'random_state': 42,
#   'verbose': 0,[6]
#   'warm_start': False})
# MSE: 2.3713904682790568
# MAE: 1.254419160074219
# R² Score: 0.5500838536218995


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df[(df['trait_TFT14_Suits'] == 3.0) & (df['placement'] == 1)]